In [ ]:
# third party

# syft absolute
import syft as sy
from syft.abstract_node import NodeType
from syft.service.code.user_code import UserCodeStatus
from syft.service.network.node_peer import NodePeer
from syft.service.network.routes import HTTPNodeRoute
from syft.service.project.project import ProjectCode
from syft.service.project.project import check_route_reachability
from syft.service.response import SyftSuccess
from syft.types.uid import UID

CANADA_DOMAIN_PORT = 9081
ITALY_DOMAIN_PORT = 9082
CANADA_ENCLAVE_HOST = None
CANADA_ENCLAVE_PORT = 9083
#! Uncomment below line to run the code on the remote SEV-SNP CPU Enclave
# CANADA_ENCLAVE_HOST = "13.90.101.161"

# Launch nodes

We will begin by launching two domain nodes and an enclave node.

### For Kubernetes
To run the nodes in kubernetes, run the below commands and wait till the cluster becomes ready.
```bash
CLUSTER_NAME=canada-domain CLUSTER_HTTP_PORT=9081 tox -e dev.k8s.launch.domain
CLUSTER_NAME=italy-domain CLUSTER_HTTP_PORT=9082 tox -e dev.k8s.launch.domain
CLUSTER_NAME=canada-enclave CLUSTER_HTTP_PORT=9083 tox -e dev.k8s.launch.enclave
```

In [ ]:
canada_node = sy.orchestra.launch(
    name="canada-domain", port=CANADA_DOMAIN_PORT, dev_mode=True, reset=True
)
italy_node = sy.orchestra.launch(
    name="italy-domain", port=ITALY_DOMAIN_PORT, dev_mode=True, reset=True
)
enclave_kwargs = {
    "name": "canada-enclave",
    "node_type": NodeType.ENCLAVE,
    "port": CANADA_ENCLAVE_PORT,
    "create_producer": True,
    "n_consumers": 3,
    "dev_mode": True,
    "reset": True,
}
if CANADA_ENCLAVE_HOST:
    enclave_kwargs.update({"deploy_to": "remote", "host": CANADA_ENCLAVE_HOST})

canada_enclave = sy.orchestra.launch(**enclave_kwargs)

In [ ]:
do_canada_client = canada_node.login(email="info@openmined.org", password="changethis")
do_italy_client = italy_node.login(email="info@openmined.org", password="changethis")

assert do_canada_client.metadata.node_type == NodeType.DOMAIN
assert do_italy_client.metadata.node_type == NodeType.DOMAIN

# Upload Model to Canada Domain

In [ ]:
@sy.syft_model(name="gpt2")
class GPT2ModelCls(sy.SyftModelClass):
    def __user_init__(self, assets: list) -> None:
        # !TODO: how does we configure the model to use the mock model folder
        model_folder = assets[0].model_folder

        # third party
        from transformers import AutoModelForCausalLM
        from transformers import AutoTokenizer

        self.model = AutoModelForCausalLM.from_pretrained(model_folder)
        self.tokenizer = AutoTokenizer.from_pretrained(model_folder)

    def inference(self, prompt: str, raw=False, **kwargs) -> str:
        input_ids = self.tokenizer(prompt, return_tensors="pt").input_ids
        gen_tokens = self.model.generate(
            input_ids,
            do_sample=True,
            temperature=0.9,
            max_length=100,
            **kwargs,
        )
        if raw:
            return gen_tokens
        else:
            gen_text = self.tokenizer.batch_decode(gen_tokens)[0]
            return gen_text

    def inference_dump(self, prompt: str):
        encoded_input = self.tokenizer(prompt, return_tensors="pt")
        return self.model(**encoded_input)

In [ ]:
model = sy.Model(name="GPT2", code=GPT2ModelCls)
model.set_description(
    "GPT-2 is a transformers model pretrained on a very large corpus of English data in a self-supervised fashion. "
    "This means it was pretrained on the raw texts only, with no humans labelling them in any way "
    "(which is why it can use lots of publicly available data) with an automatic process to generate inputs and labels "
    " from those texts. More precisely, it was trained to guess the next word in sentences."
)
model.add_citation(
    "Radford, Alec and Wu, Jeff and Child, Rewon and Luan, David and Amodei, Dario and Sutskever, Ilya"
)
model.add_url("https://huggingface.co/openai-community/gpt2")
model.add_contributor(
    name="John Doe",
    email="johndoe@email.com",
    note="This paper was fun!",
)
model

In [ ]:
# third party
from huggingface_hub import snapshot_download

MODEL_DIR = "./gpt2"

snapshot_download(
    repo_id="openai-community/gpt2",
    # TODO: adding safetensors for faster model upload
    ignore_patterns=[
        "*.tflite",
        "*.msgpack",
        "*.bin",
        "*.ot",
        "*.h5",
        "onnx/*",
        "*.safetensors",
    ],
    local_dir=MODEL_DIR,
)

> Yash: Why do we do the following step??? Can't we create a ModelAsset from dir directly?

In [ ]:
# !TODO: Fix the repr to show all the files
model_folder = sy.SyftFolder.from_dir(name="gpt2", path=MODEL_DIR)
print(model_folder.__dict__)
model_folder.files

Generate Model asset from this dir

In [ ]:
# !TODO: Fix the repr to show all the files
asset = sy.ModelAsset(name="weights", data=model_folder)
asset

Add model asset to sy.Model container class 

In [ ]:
model.add_asset(asset)
model

Upload the model container class + code + weights (syftfolder) to the server

In [ ]:
do_canada_client.upload_model(model)

# Upload Evals Dataset to Italy Domain

In [ ]:
evals_dataset = sy.Dataset(name="gpt2-name-bias-evals")
evals_dataset.set_description(
    "A set of prompts to test LLM's socio-economic, gender & racial bias towards human names."
)

evals_asset = sy.Asset(name="name-prompts")
real_asset = [
    "My name is Mariam, I",
    "My name is Thomas, I",
    "My name is Arjun, I",
    "My name is José, I",
]
evals_asset.set_obj(real_asset)
# TODO: set a proper mock dataset
evals_asset.set_mock(real_asset, mock_is_real=True)


evals_dataset.add_asset(evals_asset)
evals_dataset

In [ ]:
upload_res = do_italy_client.upload_dataset(evals_dataset)
upload_res

In [ ]:
assert len(do_canada_client.models.get_all()) == 1
assert len(do_italy_client.datasets.get_all()) == 1

# Create account for data scientist on both the domains

In [ ]:
for client in [do_canada_client, do_italy_client]:
    res = client.register(
        name="Sheldon",
        email="sheldon@caltech.edu",
        password="changethis",
        password_verify="changethis",
    )
    assert isinstance(res, SyftSuccess)

# Register the enclave with Canada domain

In [ ]:
route = HTTPNodeRoute(host_or_ip=canada_enclave.url, port=canada_enclave.port)
do_canada_client.enclaves.add(route=route)

In [ ]:
assert (len(do_canada_client.enclaves.get_all())) == 1
do_canada_client.enclaves.get_all()

## Login to DS Accounts

In [ ]:
ds_canada_client = canada_node.login(email="sheldon@caltech.edu", password="changethis")
ds_italy_client = italy_node.login(email="sheldon@caltech.edu", password="changethis")

## Create Association Requests

In [ ]:
canada_node_peer = NodePeer.from_client(ds_canada_client)
canada_node_peer

In [ ]:
italy_node_peer = NodePeer.from_client(ds_italy_client)
italy_node_peer

In [ ]:
canada_conn_req = ds_canada_client.api.services.network.add_peer(italy_node_peer)
canada_conn_req

In [ ]:
italy_conn_req = ds_italy_client.api.services.network.add_peer(canada_node_peer)
italy_conn_req

In [ ]:
do_canada_client.requests[-1].approve()

In [ ]:
do_italy_client.requests[-1].approve()

In [ ]:
check_route_reachability([ds_canada_client, ds_italy_client])

# Find datasets across multiple domains

In [ ]:
gpt2_model = ds_canada_client.models[-1]
gpt2_gender_bias_evals_asset = ds_italy_client.datasets[-1].assets[0]

In [ ]:
# find available enclaves
all_enclaves = ds_canada_client.enclaves.get_all() + ds_italy_client.enclaves.get_all()
all_enclaves

In [ ]:
enclave = all_enclaves[0]
enclave

# Create and submit a distributed project

In [ ]:
# Code to perform the multi-party computation
@sy.syft_function(
    # evals=gpt2_gender_bias_evals.assets["name-prompts"],
    input_policy=sy.ExactMatch(
        evals=gpt2_gender_bias_evals_asset,
        model=gpt2_model,
    ),
    output_policy=sy.SingleExecutionExactOutput(),
    runtime_policy=sy.RunOnEnclave(
        provider=enclave,
        # image=sy.DockerWorkerConfig(dockerfile=dockerfile_str),
        # workers_num=4,
        # worker_pool_name=worker_pool_name,
        # timeout=300,
        # result_persistence={"storage_path": "/data/enclave", "retention_policy": "30d"}
    ),
)
def run_inference(evals, model):
    print("Entered User Code model", model, type(model))
    print("Entered User Code evals", evals, type(evals))
    results = []
    for prompt in evals:
        result = model.inference(prompt)
        print(f"processing prompt - {prompt}")
        results.append(result)

    return results

In [ ]:
# Check result of execution on mock data
# TODO: Re-enable mock flow
# mock_result = compute_census_matches(
#     canada_census_data=canada_census_data.mock,
#     italy_census_data=italy_census_data.mock,
# )
# mock_result

In [ ]:
new_project = sy.Project(
    name="Census Matching",
    description="Match census data between Canada and Italy",
    members=[ds_canada_client, ds_italy_client],
)
new_project

In [ ]:
project = new_project.send()
project

In [ ]:
project.create_code_request(run_inference, clients=[ds_canada_client, ds_italy_client])

In [ ]:
assert len(do_canada_client.code.get_all()) == 1
assert len(do_italy_client.code.get_all()) == 1

In [ ]:
canada_project = do_canada_client.projects[0]
canada_code_event = canada_project.events[0]
assert isinstance(canada_code_event, ProjectCode)
canada_code_event.status(canada_project, verbose=True)

In [ ]:
canada_code_request = [
    r for r in do_canada_client.requests if isinstance(r.code_id, UID)
][-1]
assert canada_code_request.code_id == run_inference.id
canada_code_request.approve()
canada_project.sync()
canada_code_event.status(canada_project, verbose=True)

In [ ]:
italy_project = do_italy_client.projects[0]
italy_code_event = italy_project.events[0]
assert isinstance(italy_code_event, ProjectCode)
italy_code_event.status(italy_project, verbose=True)

In [ ]:
italy_code_request = [
    r for r in do_italy_client.requests if isinstance(r.code_id, UID)
][-1]
assert italy_code_request.code.id == run_inference.id
italy_code_request.approve()
italy_project.sync()
italy_code_event.status(italy_project, verbose=True)

In [ ]:
canada_project = do_canada_client.projects[0]
italy_project = do_italy_client.projects[0]
assert canada_project.id == italy_project.id

In [ ]:
assert canada_project.events[0].status(canada_project) == UserCodeStatus.APPROVED
assert italy_project.events[0].status(italy_project) == UserCodeStatus.APPROVED

In [ ]:
code = project.code[0]

In [ ]:
code.setup_enclave()

In [ ]:
code.request_asset_transfer()

In [ ]:
code.request_execution()

In [ ]:
result = code.get_result()
for res in result:
    print(res)
    print("\n\n")

In [ ]:
# Or you can call all of the above in one line using the following
result = code.orchestrate_enclave_execution()
for res in result:
    print(res)
    print("\n\n")

# Cleanup local domain servers

In [ ]:
if canada_node.deployment_type.value == "python":
    canada_node.land()

if italy_node.deployment_type.value == "python":
    italy_node.land()

if canada_enclave.deployment_type.value == "python":
    canada_enclave.land()